## Fasam - NLP Competition

Meu notebook para a competição do **Kaggle da Fasam**. Faz parte da avaliação prática dos alunos da turma de Deep Learning da Faculdade Sul Americana

### Acesso a competição

Acesso a competição: https://www.kaggle.com/t/ea2dc714f47a41a3a8c0a099bf0e8362

### Roteiro do Notebook

* Leitura do Dataset
* Criação do Modelo
* Avaliação e Criação do arquivo de submission.csv

### Problema


Uma revista precisa catalogar todas as suas notícias em diferentes categorias. O objetivo desta competição é desenvolver o melhor modelo de aprendizagem profunda para prever a categoria de novas notícias.


<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/04/Untitled-Diagram.png
" style="width: 400px;"/>


As categorias possíveis são:

* ambiente
* equilibrioesaude
* sobretudo
* educacao
* ciencia
* tec
* turismo
* empreendedorsocial
* comida


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

sns.set(style="ticks")
warnings.filterwarnings("ignore")

In [0]:
# Bibliotecas do keras
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split

## Configuração do Ambiente Colab

Instalação de dependencias, download do dataset..

In [0]:
!wget https://storage.googleapis.com/ms_geral/fasam-dl-nlp.zip
!unzip fasam-dl-nlp.zip

## Leitura do dataset de treinamento

In [175]:
# Leitura do Dataset
df = pd.read_csv('train.csv')
print(df.shape)
df.head()


(9716, 5)


,Unnamed: 0,title,text,link,category
0,0,"Plano de Energia Limpa de Obama será revogado,...","O presidente dos Estados Unidos, Donald Trump,...",http://www1.folha.uol.com.br/ambiente/2017/10/...,ambiente
1,1,Por que é mais difícil para as mulheres lutar ...,Gabriela* percebeu que precisava de ajuda quan...,http://www1.folha.uol.com.br/equilibrioesaude/...,equilibrioesaude
2,2,\n\t\tDa escola ao 1º emprego: como buscar um ...,DA BBC BRASIL O trimestre de maio a julho des...,http://www1.folha.uol.com.br/sobretudo/carreir...,sobretudo
3,3,Compare as mensalidades de 1.104 escolas priva...,Confira abaixo as mensalidades de 1.104 escola...,http://www1.folha.uol.com.br/educacao/2017/10/...,educacao
4,4,Programa espacial soviético teve ideal social ...,"É inegável que a Revolução Russa, ainda que te...",http://www1.folha.uol.com.br/ciencia/2017/10/1...,ciencia


Todos os artigos contêm o **título, descrição e link** da matéria original. Por último a categoria que pertence esse artigo.

In [0]:
## Definição de alguns parâmetros dos modelos e tokenização

# Tamanho da sequencia
seq_size     = 180

# Máximo de tokens 
max_tokens   = 3000

# Tamanho do embedding
embed_dim    = 256

Iremos utilizar o titulo para o nosso modelo baseline. O processo abaixo cria o **input** da nossa rede e prepara o **target**

In [0]:
## Utilizaremos apenas o .title (input) e o .category (target) da nossa rede
# Textos
text         = df['text'].values
tokenizer    = Tokenizer(num_words=max_tokens, split=' ')

# Transforma o texto em números
tokenizer.fit_on_texts(text)
X = tokenizer.texts_to_sequences(text)  

# Cria sequencias de tamanho fixo (input: X)
X = pad_sequences(X, maxlen=seq_size)

In [0]:
# Categoriza o target "category" -> [0,..., 1] (output: y)
Y_classes = pd.get_dummies(df['category']).columns
Y         = pd.get_dummies(df['category']).values

In [179]:
(X.shape, Y.shape)

((9716, 180), (9716, 9))

## Criação do Modelo

Iremos utilizar uma RNN em um modelo simples.

In [190]:
def base_model():
    model = Sequential()
    
    # Embedding Layer
    model.add(Embedding(max_tokens, embed_dim, 
                        input_length = seq_size))
    
    model.add(Dropout(0.5))
    
    # RNN Layer
    model.add(LSTM(seq_size))
    
    model.add(Dropout(0.6))
    
    # Dense Layer
    model.add(Dense(len(Y_classes), activation='softmax'))
    
    model.compile(loss = 'categorical_crossentropy', 
                  optimizer='adam',
                  metrics = ['accuracy'])
    
    model.summary()
    
    return model

base_model = base_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 180, 256)          768000    
_________________________________________________________________
dropout_26 (Dropout)         (None, 180, 256)          0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 180)               314640    
_________________________________________________________________
dropout_27 (Dropout)         (None, 180)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 9)                 1629      
Total params: 1,084,269
Trainable params: 1,084,269
Non-trainable params: 0
_________________________________________________________________


In [193]:
# Separa o dataset em dados de treinamento/teste
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y, 
                                                      test_size = 0.15, 
                                                      random_state = 42)

# Treina o modelo
hist = base_model.fit(X_train, Y_train, 
              validation_data =(X_valid, Y_valid),
              batch_size=512, nb_epoch = 50,  verbose = 1)

Train on 8258 samples, validate on 1458 samples
Epoch 1/50
8258/8258 [==============================] - 99s 12ms/step - loss: 2.0757 - acc: 0.2099 - val_loss: 1.9408 - val_acc: 0.3292
Epoch 2/50
8258/8258 [==============================] - 99s 12ms/step - loss: 2.0647 - acc: 0.3302 - val_loss: 2.0461 - val_acc: 0.3594
Epoch 3/50
8258/8258 [==============================] - 99s 12ms/step - loss: 1.9556 - acc: 0.3823 - val_loss: 1.8291 - val_acc: 0.4259
Epoch 4/50
8258/8258 [==============================] - 98s 12ms/step - loss: 1.6436 - acc: 0.4639 - val_loss: 1.4401 - val_acc: 0.5158
Epoch 5/50
8258/8258 [==============================] - 97s 12ms/step - loss: 1.2340 - acc: 0.5891 - val_loss: 1.1687 - val_acc: 0.6125
Epoch 6/50
8258/8258 [==============================] - 97s 12ms/step - loss: 1.0591 - acc: 0.6534 - val_loss: 1.2515 - val_acc: 0.5741
Epoch 7/50
8258/8258 [==============================] - 98s 12ms/step - loss: 0.9964 - acc: 0.6676 - val_loss: 0.9902 - val_acc: 0.6742


## Avaliação e Criação do arquivo de submission.csv

In [194]:
# Avaliação do modelo para o dataset de test

val_loss, val_acc = base_model.evaluate(X_valid, Y_valid)

print('A acurácia do modelo está de: '+str(val_acc*100)+'%')

1458/1458 [==============================] - 6s 4ms/step
A acurácia do modelo está de: 81.1385459288321%


### Criando arquivo de submission para o Kaggle

In [195]:
# Leitura do Dataset de validação dos resultados
df_valid = pd.read_csv('valid.csv')
print(df_valid.shape)
df_valid.head()

(2429, 3)


,Unnamed: 0,title,text
0,0,Anticorpo artificial pode eliminar vírus HIV d...,Pesquisadores anunciaram no mês passado algo q...
1,1,Inscrições para o Enem 2015 começam nesta segu...,Estudantes interessados em participar do Enem ...
2,2,"Saúde responde: Como diferenciar dengue, chiku...",Gostaria de saber como é possível diferenciar ...
3,3,Cervejarias artesanais e bares especializados ...,No porão de um prédio residencial ao norte de ...
4,4,"Contra votação do Plano de Educação, grupo pro...",Um grupo de manifestantes protesta em frente à...


O dataset de validação, o que será utilizado para calcular o Ranking no Kaggle, contêm apenas as informações de Título e Texto do arquivo.  O modelo criado deve ser capaz de classificar em qual das categorias esse artigo pertence

In [0]:
def predict(text):
    '''
    Utiliza o modelo treinado para realizar a predição
    '''
    new_text = tokenizer.texts_to_sequences(text)
    new_text = pad_sequences(new_text, maxlen=seq_size)
    pred     = base_model.predict_classes(new_text)#[0]
    return pred

In [197]:
# Como utilizamos o titulo no treinamento, iremos utilizar o titulo na predição também

pred         = predict(df_valid.text)
pred_classes = [Y_classes[c] for c in pred]
pred_classes[:5]

['ciencia', 'educacao', 'equilibrioesaude', 'comida', 'ciencia']

In [198]:
# Atualizando a categoria dos artigos no dataset de validação
df_valid['category'] = pred_classes
df_valid.head()

,Unnamed: 0,title,text,category
0,0,Anticorpo artificial pode eliminar vírus HIV d...,Pesquisadores anunciaram no mês passado algo q...,ciencia
1,1,Inscrições para o Enem 2015 começam nesta segu...,Estudantes interessados em participar do Enem ...,educacao
2,2,"Saúde responde: Como diferenciar dengue, chiku...",Gostaria de saber como é possível diferenciar ...,equilibrioesaude
3,3,Cervejarias artesanais e bares especializados ...,No porão de um prédio residencial ao norte de ...,comida
4,4,"Contra votação do Plano de Educação, grupo pro...",Um grupo de manifestantes protesta em frente à...,ciencia


In [0]:
def create_submission(df):
    f = open('submission_valid.csv', 'w')
    f.write('ID,category\n')
    for i, row in df.iterrows():
        f.write('{},{}\n'.format(i, row.category))
    f.close()

In [0]:
# Criando o arquivo submission_valid.csv contendo os dados para cálculo do raning no kaggle
# Esse arquivo deve ser enviado para o kaggle
create_submission(df_valid)

In [0]:
from google.colab import files

# download submission_valid.csv
files.download('submission_valid.csv') 